In [2]:
import os
import torch
import numpy as np
from tqdm import tqdm
import optuna
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random
from pytorch_tabnet.tab_model import TabNetRegressor
from scipy import stats
import re
from pandas.core.frame import DataFrame
import warnings
warnings.filterwarnings("ignore")

In [2]:
def calc_metrics(y_pred, y_test):
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = metrics.mean_squared_error(y_test, y_pred, squared=False)
    r_2 = metrics.r2_score(y_test, y_pred)
    mae = metrics.mean_absolute_error(y_test, y_pred)
    pcc, _ = stats.pearsonr(y_test, y_pred)
    return [mse, rmse, r_2, mae, pcc]

In [33]:
def average_predict(x: np.array, all_models: list) -> float:
    all_pred = []

    for model in all_models:
        pred = model.predict(x)
        all_pred.append(pred)

    all_pred = np.concatenate(all_pred, axis=1)
    tm = all_pred.mean(axis=1).flatten()
    return tm

In [3]:
def print_metrics(**kwargs):
    metrics = pd.DataFrame(kwargs, index = [0])
    return metrics

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
def process_repr(repr):
    digits = re.findall(r'[-+]?\d*\.?\d+', repr)
    float_repr = [float(item) for item in digits]
    return float_repr

In [22]:
train = pd.read_csv('../data/train.csv')
full_test = pd.read_csv('../data/blind_test.csv')


In [23]:
train['repr'] = train['repr'].apply(process_repr)
full_test['repr'] = full_test['repr'].apply(process_repr)


In [19]:
SEED = 42
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

In [20]:
def process_features(chunk: DataFrame) -> tuple[np.ndarray, np.ndarray]:
    x = chunk.loc[:, 'repr']
    x = np.array(list(x))
    y = chunk.loc[:, 'Tm']
    y = np.array(list(y)).reshape(-1, 1)
    return x, y

In [23]:
x, y = process_features(train)

In [36]:
x_test, y_test = process_features(full_test)

In [24]:
X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.33, random_state=SEED)

In [21]:
MAX_EPOCHS = 300
PATIENCE = 30
BATCH_SIZE = 1024

##### Perform hyperparameters search

In [68]:
def objective(trial):
    n_shared = trial.suggest_int('n_shared', 1, 5)
    n_independent = trial.suggest_int('n_independent', 1, 5)
    n_d = trial.suggest_int('n_d', 8, 64)
    n_a = trial.suggest_int('n_a', 8, 64)
    n_steps = trial.suggest_int('n_steps', 3, 10)
    gamma = trial.suggest_float('gamma', 1.0, 2.0)
    momentum = trial.suggest_float('momentum', 0.1, 0.9)
    lambda_sparse = trial.suggest_loguniform('lambda_sparse', 1e-5, 1e-1)
    
    
    model = TabNetRegressor(n_d=n_d, n_a=n_a, n_independent=n_independent, n_shared=n_shared, n_steps=n_steps, gamma=gamma, momentum=momentum, 
                            lambda_sparse=lambda_sparse, verbose=0, seed=SEED, device_name=device, optimizer_fn=torch.optim.Adam, 
                            scheduler_fn=torch.optim.lr_scheduler.StepLR, scheduler_params={'step_size': 10, 'gamma': 0.9})

    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], patience=PATIENCE, max_epochs=MAX_EPOCHS, batch_size=BATCH_SIZE)
    y_pred = model.predict(X_valid)
    mse = metrics.mean_squared_error(y_valid, y_pred)
    return mse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=70)
for i, trial in enumerate(study.trials):
    print(f'Iteration {i+1}: Best value = {study.best_value}')

[I 2023-07-01 05:05:17,398] A new study created in memory with name: no-name-69076c3a-bea7-4304-af82-59d1a219d28e



Early stopping occurred at epoch 114 with best_epoch = 84 and best_val_0_mse = 37.45805


[I 2023-07-01 05:12:11,056] Trial 0 finished with value: 37.45805462099855 and parameters: {'n_shared': 1, 'n_independent': 3, 'n_d': 32, 'n_a': 53, 'n_steps': 10, 'gamma': 1.5843747996502784, 'momentum': 0.11172385993515209, 'lambda_sparse': 0.05430249254142419}. Best is trial 0 with value: 37.45805462099855.



Early stopping occurred at epoch 103 with best_epoch = 73 and best_val_0_mse = 37.71494


[I 2023-07-01 05:17:25,260] Trial 1 finished with value: 37.71494182874437 and parameters: {'n_shared': 3, 'n_independent': 3, 'n_d': 44, 'n_a': 32, 'n_steps': 6, 'gamma': 1.0515222749271895, 'momentum': 0.5236862805850789, 'lambda_sparse': 0.007550418166087127}. Best is trial 0 with value: 37.45805462099855.


...

[I 2023-07-01 07:55:29,243] Trial 32 finished with value: 37.493893942834625 and parameters: {'n_shared': 2, 'n_independent': 3, 'n_d': 46, 'n_a': 41, 'n_steps': 6, 'gamma': 1.5819525171714077, 'momentum': 0.28065503789406565, 'lambda_sparse': 0.015536465100099687}. Best is trial 4 with value: 37.21355574017806.



Early stopping occurred at epoch 111 with best_epoch = 81 and best_val_0_mse = 37.87822


[I 2023-07-01 08:01:59,817] Trial 33 finished with value: 37.87822294126423 and parameters: {'n_shared': 2, 'n_independent': 2, 'n_d': 34, 'n_a': 30, 'n_steps': 10, 'gamma': 1.6186655108983905, 'momentum': 0.1636118134982978, 'lambda_sparse': 0.0021376885172221106}. Best is trial 4 with value: 37.21355574017806.



Early stopping occurred at epoch 103 with best_epoch = 73 and best_val_0_mse = 38.22591


[I 2023-07-01 08:06:05,143] Trial 34 finished with value: 38.225914438927155 and parameters: {'n_shared': 1, 'n_independent': 3, 'n_d': 41, 'n_a': 49, 'n_steps': 6, 'gamma': 1.5262080621617142, 'momentum': 0.2520215535799848, 'lambda_sparse': 5.103762564680734e-05}. Best is trial 4 with value: 37.21355574017806.


#### Trial 4 has the best score

##### Make stratified Grouped cross-validation accounting group Tm and evloutional cluster

In [23]:
from sklearn.model_selection import StratifiedGroupKFold

In [24]:
sgkf = StratifiedGroupKFold(n_splits=10)


In [26]:
X = train['repr'].values
y = train['group_Tm'].values
groups = train['clst'].values

In [27]:
from collections import defaultdict

In [28]:
num_fold = defaultdict(list)

In [29]:
for i, (train_idx, test_idx) in enumerate(sgkf.split(X, y, groups=groups)):
    num_fold[i].append(test_idx)

In [30]:
train['num_fold'] = [None] * len(train)

In [31]:
for key in num_fold:
    train['num_fold'].iloc[num_fold[key]] = key

In [32]:
full_test

,Protein ID,Tm,repr
0,Q72HG4_TT_C1523,74.007,"[0.094343215, -0.13420974, 0.018577041, -0.116..."
1,Q745T7_TT_P0220,77.065,"[0.0594677, -0.18177457, -0.013549126, -0.1414..."
2,Q72G97_recG,72.153,"[0.11095836, -0.09452299, 0.024978561, -0.1432..."
3,Q745Z3_TT_P0162,73.594,"[0.05716914, -0.10307638, 0.007367082, -0.0030..."
4,Q72HN7_TT_C1449,79.211,"[0.014693282, -0.11982835, -0.031532157, -0.09..."
...,...,...,...
1195,P54460_prmA,46.766,"[0.03920728340744972, -0.046484656631946564, 0..."
1196,O89001_Cpd,49.077,"[0.012924259528517723, -0.10131210833787918, 0..."
1197,P46019_PHKA2,51.235,"[0.04998369514942169, -0.05574483424425125, -0..."
1198,Q9HCN4-2_GPN1,60.556,"[0.022666864097118378, -0.08774963021278381, -..."


In [33]:
train.num_fold.value_counts()

8    2866
4    2866
1    2866
6    2866
5    2866
0    2866
3    2866
2    2866
9    2865
7    2865
Name: num_fold, dtype: int64

Trial 4 finished with value: 37.21355574017806 and parameters: {'n_shared': 5, 'n_independent': 5, 'n_d': 63, 'n_a': 12, 'n_steps': 5, 'gamma': 1.4111484359834034, 'momentum': 0.3592637810142343, 'lambda_sparse': 1.2038760187881758e-05}. Best is trial 4 with value: 37.21355574017806.

In [34]:
models = []
valid_metrics = []
for fold in tqdm(train['num_fold'].unique()):
    train_sample = train[train['num_fold']!=fold]
    valid_sample = train[train['num_fold']==fold]
    x_train, y_train = process_features(train_sample)
    x_valid, y_valid = process_features(valid_sample)
    model = TabNetRegressor(n_d=63, n_a=12, n_independent=5, n_shared=5, n_steps=5, gamma=1.411, momentum=0.35926, 
                            lambda_sparse=1.20e-5, verbose=0, seed=SEED, device_name=device, optimizer_fn=torch.optim.Adam, 
                            scheduler_fn=torch.optim.lr_scheduler.StepLR, scheduler_params={'step_size':10, 'gamma':0.9})
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], patience=PATIENCE, max_epochs=MAX_EPOCHS, batch_size=BATCH_SIZE)
    y_pred = model.predict(x_valid)
    y_pred = y_pred.flatten()
    y_valid = y_valid.flatten()
    metrics_value = calc_metrics(y_pred, y_valid)
    print(metrics_value)
    valid_metrics.append(metrics_value)
    models.append(model)

  0%|          | 0/10 [00:00<?, ?it/s]


Early stopping occurred at epoch 72 with best_epoch = 42 and best_val_0_mse = 39.11485


 10%|█         | 1/10 [05:28<49:19, 328.85s/it]

[39.114846683851006, 6.254186332677578, 0.6772552140056775, 4.572483316343497, 0.8260796761525663]

Early stopping occurred at epoch 71 with best_epoch = 41 and best_val_0_mse = 39.42174


 20%|██        | 2/10 [10:57<43:50, 328.79s/it]

...

[36.611652527653774, 6.050756359964742, 0.6891468899667648, 4.467581531035862, 0.830356273288147]

Early stopping occurred at epoch 78 with best_epoch = 48 and best_val_0_mse = 36.38061


 90%|█████████ | 9/10 [49:47<05:31, 331.41s/it]

[36.380612614186155, 6.031634323646134, 0.7046559053120582, 4.499017571711165, 0.8403641922081609]

Early stopping occurred at epoch 66 with best_epoch = 36 and best_val_0_mse = 37.77945


100%|██████████| 10/10 [54:51<00:00, 329.17s/it]

[37.77945273503293, 6.146499225984897, 0.68708594555277, 4.644880397782985, 0.8294169300944408]


In [38]:
valid_metrics = np.array(valid_metrics)

In [41]:
mean_val_metrics = valid_metrics.mean(axis=0)

In [43]:
std_val_metrics = valid_metrics.std(axis=0)

In [50]:
val_metrics = pd.DataFrame({'mean': mean_val_metrics, 'std': std_val_metrics}, index=['mse', 'rmse', 'r_2', 'mae', 'pcc']).T

In [51]:
val_metrics

,mse,rmse,r_2,mae,pcc
mean,37.562521,6.128057,0.689629,4.556162,0.831369
std,1.190984,0.097153,0.008615,0.070101,0.004811


In [55]:
ytest_pred = average_predict(x_test, models)

In [58]:
y_test = y_test.flatten()

In [59]:
mse, rmse, r_2, mae, pcc = calc_metrics(ytest_pred, y_test)

In [60]:
print_metrics(mse=mse, mae=mae, rmse=rmse, r_2=r_2, pcc=pcc)

,mse,mae,rmse,r_2,pcc
0,33.234517,4.257874,5.764939,0.70705,0.840879


#### Save models

In [62]:
for i in range(len(models)):
    torch.save(models[i], f'../model/tabnet_model{i}.pt')